In [1]:
import nltk
import random
# from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
import pickle

from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

from nltk.classify import ClassifierI
from statistics import mode

from nltk.tokenize import word_tokenize
from unidecode import unidecode
import codecs

### Created new VoteClassifier class

In [2]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

### Data Preprocessing

In [3]:
# load data
# short_pos = open("short_reviews/positive1.txt","r").read()
# short_neg = open("short_reviews/negative1.txt","r").read()
short_pos = codecs.open("short_reviews/positive1.txt", encoding='utf-8').read()
short_neg = codecs.open("short_reviews/negative1.txt", encoding='utf-8').read()

In [4]:
# move this up here
all_words = []
documents = []


#  j is adject, r is adverb, and v is verb
#allowed_word_types = ["J","R","V"]
allowed_word_types = ["J"]

for p in short_pos.split('\n'):
    documents.append( (p, "pos") )
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())

    
for p in short_neg.split('\n'):
    documents.append( (p, "neg") )
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())

### Save Documents

In [7]:
save_documents = open("pickled_algos/documents.pickle","wb")
pickle.dump(documents, save_documents)
save_documents.close()

### Get most frequent 5000 words

In [8]:
all_words = nltk.FreqDist(all_words)

word_features = list(all_words.keys())[:5000]

### Save Word Features

In [9]:
save_word_features = open("pickled_algos/word_features5k.pickle","wb")
pickle.dump(word_features, save_word_features)
save_word_features.close()

### Find features

In [10]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

### Create all feature sets

In [11]:
featuresets = [(find_features(rev), category) for (rev, category) in documents]

In [26]:
save_featuresets = open("pickled_algos/featuresets.pickle","wb")
pickle.dump(featuresets, save_featuresets)
save_featuresets.close()

In [12]:
# Shuffle all sets
random.shuffle(featuresets)
print(len(featuresets))

10664


### Split into Train and Test set

In [13]:
training_set = featuresets[:10000]
testing_set =  featuresets[10000:]

### Save Original Naive Bayes

In [14]:
classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

save_classifier = open("pickled_algos/originalnaivebayes5k.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

('Original Naive Bayes Algo accuracy percent:', 70.93373493975903)
Most Informative Features
                 generic = True              neg : pos    =     16.4 : 1.0
                mediocre = True              neg : pos    =     15.1 : 1.0
               inventive = True              pos : neg    =     14.9 : 1.0
                 routine = True              neg : pos    =     14.4 : 1.0
                    flat = True              neg : pos    =     14.3 : 1.0
               wonderful = True              pos : neg    =     12.5 : 1.0
                powerful = True              pos : neg    =     12.1 : 1.0
                   stale = True              neg : pos    =     11.7 : 1.0
                mindless = True              neg : pos    =     11.1 : 1.0
                  stupid = True              neg : pos    =     11.1 : 1.0
               realistic = True              pos : neg    =     10.9 : 1.0
                    dull = True              neg : pos    =     10.9 : 1.0
       

### Save Multinomial Naive Bayes

In [15]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

save_classifier = open("pickled_algos/MNB_classifier5k.pickle","wb")
pickle.dump(MNB_classifier, save_classifier)
save_classifier.close()

('MNB_classifier accuracy percent:', 69.27710843373494)


### Save Bernoulli Naive Bayes

In [16]:
BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

save_classifier = open("pickled_algos/BernoulliNB_classifier5k.pickle","wb")
pickle.dump(BernoulliNB_classifier, save_classifier)
save_classifier.close()

('BernoulliNB_classifier accuracy percent:', 69.7289156626506)


### Save LogisticRegression

In [17]:
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

save_classifier = open("pickled_algos/LogisticRegression_classifier5k.pickle","wb")
pickle.dump(LogisticRegression_classifier, save_classifier)
save_classifier.close()

('LogisticRegression_classifier accuracy percent:', 68.37349397590361)


### Save SGDClassifier_classifier

In [18]:
SGDC_classifier = SklearnClassifier(SGDClassifier())
SGDC_classifier.train(training_set)
print("SGDClassifier accuracy percent:",nltk.classify.accuracy(SGDC_classifier, testing_set)*100)

save_classifier = open("pickled_algos/SGDC_classifier5k.pickle","wb")
pickle.dump(SGDC_classifier, save_classifier)
save_classifier.close()

('SGDClassifier accuracy percent:', 68.07228915662651)


### Save LinearSVC_classifier

In [19]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

save_classifier = open("pickled_algos/LinearSVC_classifier5k.pickle","wb")
pickle.dump(LinearSVC_classifier, save_classifier)
save_classifier.close()

('LinearSVC_classifier accuracy percent:', 68.37349397590361)
